
# Práctica 2: Metaheurísticas basadas en trayectorias: Tabu Search

<center><h3>
    Uxía Carro Tacón
</h3></center>


# Instrucciones

Igual que en la Práctica 1, utilizaremos un **Jupyter Notebook** para la resolución de esta práctica.

Como ya sabéis, nos permite ir ejecutando celdas de código poco a poco, así como generar automáticamente un informe bien formateado de la práctica. Aun así, a continuación tenéis unas breves instrucciones sobre como funciona:

* Puedes añadir una celda con el botón **"Insert"** de la barra de herramentas, y cambiar su tipo con **"Cell > Cell Type"**
* Para ejecutar una celda de código, la seleccionaremos y pulsaremos el botón **"▶ Run"** de la barra de herramentas.
* Para pasar el documento a HTML, seleccionaremos **"File > Download as > HTML (.html)"**

Sigue este guión hasta el final. Ejecuta el código proporcionado paso a paso comprendiendo lo que estás haciendo y reflexionando sobre los resultados. Habrá preguntas intercaladas a lo largo del guión, responde a todas ellas en la sección reservada para ese fin: **"Respuestas a los cuestionarios"**. Por favor, no modifiques ninguna linea de código excepto cuando se te pida explícitamente.

No olvides insertar tu **nombre y apellidos** en la celda superior.

IMPORTANTE: Se te pedirán dos implementaciones del algoritmo de Búsqueda Tabú, una primera implementación obligatoria y una implementación mejorada optativa. Escribe el código de tu o tus soluciones en las celdas que se indican para ello. Además, a lo largo de la práctica se plantearán varias preguntas que debéis responder en la parte inferior del documento, incluyendo las celdas que veáis necesarias (si hacéis referencia a partes concretas de vuestro código, etc) para reponder a ellas.

## Entrega de la práctica

La fecha límite de entrega será la indicada en el Campus Virtual. La entrega consistirá de un único archivo comprimido con nombre `APELIDOS_NOME_BusquedaTabu.zip` que contenga los seguientes ficheros:

 * `APELIDOS_NOME_BusquedaTabu.html`: Archivo HTML fruto de la exportación del presente Notebook, con las preguntas respondidas al final del documento.
 * `APELIDOS_NOME_BusquedaTabu.ipynb`: Archivo fuente Jupyter Notebook.
 * Archivo de datos de los problema utilizados en la resolución.
 
 ---


# Preliminares adicionales sobre Python

Además de lo visto en las prácticas anteriores, conviene familiarizarse con algunas funciones disponibles en Python que pueden resultarte útiles más adelante en la realización de esta práctica y en el uso del lenguaje en general.


Por ejemplo, cuando necesitas consultar documentación sobre paquetes Python, conviene que conozcas la versión que estás utilizando en tu entorno para poder encontrar las especificidades de cada *release*. Fíjate que en la sección *Docs by version* de la web https://docs.python.org/3/ puedes seleccionar los documentos de referencia de la versión.

Para averiguar la versión que está ejecutando este Jupyter Noteboook puedes emplear las siguientes líneas.

In [3]:
from platform import python_version

print(python_version())


3.10.12


Un paquete interesante puede ser statistics. Conociendo la versión, puedes puedes consultar la documentación y la lista completa de funciones disponibles: https://docs.python.org/3/library/statistics.html

A modo ilustrativo, puedes obtener estadísticos sobre series de datos de la siguiente forma.


In [28]:
import statistics

# ejemplo de obtención de medias y desviaciones típicas
listav = [1, 2, 3, 4, 5]
media = statistics.mean(listav)
desvtip = statistics.stdev(listav)
print(media, desvtip)

listav = [1.0, 2.5, 3.75, 4.25, 5]
media = statistics.mean(listav)
desvtip = statistics.stdev(listav)
print(media, desvtip)


3 1.5811388300841898
3.3 1.5751984002023365


Otro paquete que puede resultarte útil a la hora de crear un informe o documentar tu práctica puede ser `matplotlib.pyplot`, que permite hacer gráficas de manera integrada en el propio Notebook. Puedes consultar sus capacidades en https://matplotlib.org/stable/gallery/index.html

Veámos un ejemplo ilustrativo.

In [5]:
# Indicamos al Notebook que queremos figuras interactivas (esto sólo es necesario hacerlo una vez)
%matplotlib notebook

import matplotlib.pyplot as plt  # Importamos la librería (esto sólo es necesario hacerlo una vez)
import math

# definimos algunas líneas de tendencia arbitrarias
vectorx = [x for x in range (1,10)]
vectorlogy = [ math.log(i) for i in vectorx ]
vectorpowy = [ math.pow(i, 2) for i in vectorx ]

fig = plt.figure()  # Creamos una figura (contenedor para elementos a dibujar)
ax = fig.add_subplot(111, projection='rectilinear')  # añadimos subplot-canvas a la figura (contiene los elementos del dibujo)
 
# añadimos los plots
ax.plot(vectorx, vectorlogy, '--', color="green")
ax.plot(vectorx, vectorpowy, '--', color="red")
 
ax.set_title("Ejemplo")  # Ponemos un título
ax.set_xlabel("Eje X")  # Nombramos los ejes
ax.set_ylabel("Eje Y")
 
plt.show()  # Mostramos a figura por pantalla, que se corresponde con la fig creada arriba

<IPython.core.display.Javascript object>

---

# El Problema del Viajante de Comercio (VC) con Búsqueda Tabú

De nuevo, trataremos de resolver el problema del Viajante de Comercio, pero ahora con el algoritmo de Búsqueda Tabú.

El objetivo de esta práctica es modelar e implementar un agente inteligente que sea capaz de resolver el problema del VC mediante la metaheurística (MH) de Busquedá Tabú (TS, del inglés Tabu Search). Para ello, realizarás una implementación del algoritmo básico visto en la clase expositiva y valorarás si la introducción de modificaciones en el diseño del algoritmo te permite mejorar la calidad de las soluciones alcanzadas.


## Definición del problema de Viajante de Comercio (VC)



El problema del viajante de comercio (VC) es el problema de la persona que quiere vender un producto, y que para ello quiere encontrar el viaje más corto posible a través de las ciudades de los clientes, haciendo una única visita a cada una, empezando y acabando el recorrido en su propia ciudad (recorrido circular desde la ciudad inicial).
Típicamente, el problema parte de una representación mediante un grafo ponderado $G=(N, A)$, donde $N$ es el conjunto de $n=|N|$ nodos (ciudades), y siendo A el conjunto de arcos conectando los nodos. Cada arco $(i, j) ∈ A$ tiene asignado un peso $d_ij$ que representa la distancia entre las ciudades $i$ y $j$.
El VC se reduce al problema de crear el circuito Hamiltoniano de longitud mínima sobre el grafo $G$. La solución a una instancia del problema del VC puede representarse como una permutación de los índices de las ciudades, donde lo importante es el orden de visita, que determinará el coste del viaje en términos de la distancia recorrida total. 
De este modo, el problema pertenece a la categoría de problemas NP, pues puede haber n permutaciones que se corresponden al espacio de búsqueda posible. Esto hace que resolver instancias de problemas con muchas ciudades (n grande) haga el problema impracticable con estrategias de búsqueda no-informadas y éste pueda beneficiarse de ciertas metaheurísticas, pudiendo abordar de problemas con tallas más grande a la vez que se obtienen soluciones razonablemente buenas.


### Nociones previas

Para facilitar vuestra labor de implementación, os proporcionamos la clase `Localizaciones`, que permite cargar las localizaciones GPS que representan los vértices del grafo G de N ciudades, y permite calcular de manera transparente la distancia entre cualquier par de ciudades usando la [fórmula del semiverseno]( https://es.wikipedia.org/wiki/F%C3%B3rmula_del_semiverseno), que sirve para calcular las distancias teniendo en cuenta la curvatura de la Tierra. 
Es importante tener en cuenta que en la fórmula del semiverseno las coordenadas se expresan en radianes.


En primer lugar importa el módulo Python que acompaña esta práctica, que trae alguna función de apoyo implementada así como la clase de carga de datos.

In [15]:
from helpers_mod_sa import *

Inspecciona el código de carga de localizaciones mediante `psource(Localizaciones)`

In [16]:
psource (Localizaciones)

Fíjate que por defecto se carga el fichero `./data/grafo8cidades.txt`, que contiene las coordenadas GPS de 8 ciudades gallegas, siendo Santiago de Compostela la primera de ellas. La primera línea de estos ficheros indica el número de ciudades n, mientras que cada una de las líneas sucesivas especifican las coordenadas de cada ciudad, especificadas como coordenadas GPS (latitud y longitud en grados).

Puedes cargar otro fichero haciendo uso del parámetro `filename` como se muestra a continuación. Si todo va bien, la primera distancia entre la ciudad 0 y 1 debe ser unos 55 km.

❗ Para esta práctica, **debéis utilizar** el fichero `./data/grafo100cidades.txt` que contiene las coordenadas de 100 concellos gallegos.

In [7]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))
g2=Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
print (g2.distancia(0,1))
g3=Localizaciones(filename='./data/grafo100cidades.txt')
print(g3.distancia(0,1))

55.88273580792048
119.30959564041359
68.81748609463234



## P2.1: Implementación básica de Búsqueda Tabú (especificación obligatoria, 6pts)



En este apartado debes desarrollar una **versión básica del algoritmo de Búsqueda Tabú que resuelva el problema del viajante de comercio (TSP)** aplicado a los concellos de Galicia. La especificación del algoritmo será muy detallada, ya que el objetivo principal de esta primera parte es que dispongas de una implementación totalmente funcional y verificada que resuelva el problema correctamente.
Igual que en la práctica anterior (P1), consideramos que el recorrido es circular (empieza y termina en el mismo concello) y que tienen que pasar por N=100 concellos de Galicia. Implementa el algoritmo básico de Búsqueda Tabú para resolver el problema del VC enunciado arriba. Para ello, revisa la descripción algorítmica de la MH vista en la clase expositiva.

Ten en cuenta las **siguientes consideraciones** de diseño para completar la implementación básica:
- **Representación  de  las  soluciones:**  representación  de  orden  (permutaciones)  **comenzando y finalizando en la ciudad 0**. Es decir, utilizamos una representación de orden formada por una secuencia de valores numéricos que representan cada uno de los concellos {0, 1, ..., 99}. Consideramos siempre como punto de partida y retorno el concello 0, por lo que efectivamente una solución *S* se representa como una permutación de los demás valores {1, ..., 99}. 


- **Solución inicial:** generación **totalmente aleatoria** de una permutación válida como está explicado en los apartados previos. 

❗  **NOTA: Es importante que la generación de la solución inicial se implemente en una función propia y que después, esta solución inicial se pase a la función que implementa el algoritmo de búsqueda tabú *per se*.**


- **Operador  de  generación de vecindario de una solución** (generación de la solución  siguiente  $S_{cand}$  a  partir  de  la  actual  $S_{act}$): Se utilizará el **operador  de  intercambio** y se generarán *todos los posibles intercambios**. Es decir, con este operador, se explota el vecindario de forma completa, con lo que se genera el siguiente número de vecinos:

$$ \sum_{i=1}^{L-1}i = \frac{L(L-1)}{2} $$

donde *L* es la longitud de la solución y se ha tenido en cuenta que es lo mismo intercambiar los vecinos *i, j* que *j, i*.

❗  **NOTA: Es importante que la generación del vecindario de una solución se implemente en una función propia y que después, el vecino seleccionado, se pase a la función que implementa el algoritmo de búsqueda tabú *per se*.**



- **Función de coste:** suma de las distancias del camino según el orden del recorrido teniendo en cuenta que se parte de y se llega al concello 0.  La distancia se calcula teniendo en cuenta los siguientes tres elementos:
    - Distancia desde el concello 0 al primer concello de la solución: 0 -> S[0]
    - Distancia recorrida en la solución: S[0] -> S[1] -> ... -> S[-1]
    - Distancia recorrida desde el concello final al concello 0: S[-1] ->0
    


- **Lista Tabú:** La lista tabú (LT) estará formada por los **movimientos de intercambio de índices** {i,j} que den lugar a las soluciones que forman la trayectoria de búsqueda. Tenéis que establecer **N=100** como parámetro de tenencia tabú, esto es, el tamaño de la lista tabú será de **N** elementos, de forma que un movimiento {i,j} saldrá de la lista tabú después de **N=100** operaciones y volverá a estar permitido.


- **Reinicialización:** En el caso de que transcurran **1000 iteraciones consecutivas** sin que mejore la solución óptima $S_{opt}$ alcanzada hasta el momento, se hará una reinicialización desde ella: $S_{opt}$. Se trata por tanto de una estrategia de reinicialización por **intensificación**. En el reinicio **no se restaura la lista tabú**, para dar opción a visitar vecinos no visitados anteriormente ya que determinados intercambios prohibidos al estar incluidos en la lista tabú. Es decir, cuando se hace un reinicio, se vuelve a la solución $S_{opt}$ y **se vacía la lista tabú.**


- **Criterio de parada:** Se finaliza la ejecución cuando se alcanzan **10.000 iteraciones** del algoritmo.



### Preguntas sobre la especificación básica obligatoria (se responde al final del notebook)

❓ **Pregunta 1**. Explica brevemente los detalles relevantes de tu código para entender tu implementación (p.ej., estructura de tu código, funciones, etc.)

❓ **Pregunta 2**. La parte experimental de la práctica consiste en realizar **10 ejecuciones diferentes** de la implementación realizada y reportar:
- **Media y desviación** estándar de la mejor solución obtenida.
- El **número de iteración** en el que se obtuvo la mejor solución (por lo que debéis mantener además de la solución óptima hasta el momento $S_{opt}$, la iteración en la que se obtiene). 
- El **tiempo de ejecución** del algoritmo (en las nociones previas se explica cómo hacer esto).

 
## P2.2: Mejoras del algoritmo de Búsqueda Tabú (especificación opcional, 4pts)
En este apartado el objetivo es aplicar la resolución del problema que acabáis de programar a un nuevo conjunto de 120 localizaciones tomadas del archivo de [50.000 lugares históricos del Registro Nacional de los EE.UU](http://www.math.uwaterloo.ca/tsp/us/data.html), tal y como se describe en la web del [Traveller Salesman Problem (TSP)](http://www.math.uwaterloo.ca/tsp/) del [Department of Combinatorics and Optimization](https://uwaterloo.ca/combinatorics-and-optimization/) de la University of Waterloo CA [(Prof. William Cook)](http://www.math.uwaterloo.ca/~bico/).
Para evitar problemas de tiempo excesivo de cómputo, reduciremos el problema a 120 localizaciones que se indican en el fichero **US120.txt**.

**NOTA:** Si alguno de vosotros quiere realizar pruebas con todos los lugares indicados, puede obtener el fichero de texto original en el [siguiente enlace](http://www.math.uwaterloo.ca/tsp/us/files/us50000_latlong.txt).


En esta sección el objetivo es realizar mejoras al algoritmo desarrollado previamente, de acuerdo a lo visto en las clases expositivas. Podrá modificarse cualquier parámetro u operador, como por ejemplo:
- **Generación de la solución inicial** (inicialización greedy, ...)
- **Gestión de la lista tabú** con la inclusión de algún criterio de aspiración (por ejemplo excluir de la lista una solución si mejora a la mejor solución hasta el momento)
- El **operador de generación de vecinos** (por ejemplo no considerar todos los pares de índices, cambiar el operador de generación, ...)
- Utilizar **otras estrategias de reinicialización** por intensificación (por ejemplo reiniciar desde una solución aleatoria de un conjunto de las N mejores soluciones hasta ahora, restaurar la lista tabú, ...).
- Cambiar la **estrategia de reinicialización para dar diversificación** con una estrategia de **memoria a largo plazo**. Como puede ser utilizar una matriz simétriza *frec* que almacene elnúmero de veces que cada par de ciudades han sido consecutivas en las soluciones aceptadas hasta el momento. Con esta matriz de frecuencias se podría realizar una inicialización voraz sobre una matriz de distancias modificada que incluya las frecuencias almacenadas en memoria y penalice los pares de ciudads con mayor frecuencia, incrementando ficticiamente su distancia:

$$ D(i,j)_{MOD} = D(i,j) + \mu (D_{MAX} - D_{min}) \frac{frec(i,j)}{frec_{MAX}} $$

- Utilizar un criterio de **oscilación estratégica** que alterne entre las estrategias de intensificación y diversificación**-

### Preguntas sobre la práctica optativa con las mejoras (se responde al final del notebook)

❓ **Pregunta 3**. ¿Qué intervenciones de mejora te ha llevado a mejores resultados? Explica brevemente las mejoras o intervenciones de mejora realizadas, cómo la has implementado, porqué las consideras buenas para el problema y presenta tus conclusiones acompañadas de los resultados obtenidos.


---

# Respuestas a las preguntas y evaluación

**Recordatorio:** No olvides escribir tu nombre y apellidos en la segunda celda de este documento.
La respuestas a las preguntas deben venir acompañadas de las implementaciones necesarias para su respuesta.

## P2.1 Especificación obligatoria (6pt)

La implementación básica se evaluará mediante un cuestionario automático de evaluación. Es también necesaria **realizar la implementación del algoritmo** y **responder a las preguntas 1 y 2** respectivamente. El cuestionario de evaluación lo realizarás en la primera sesión de la próxima práctica, y se centrará en la resolución por tu parte de diversas cuestiones prácticas relacionadas con la implementación realizada, pudiendo ser necesaria la ejecución, adaptación y modificación de la misma.

Aclaración: Independientemente del cuestionario automático de evaluación, siempre considera que las preguntas planteadas en el notebook deben ser respondidas también. Esas preguntas generales están diseñadas para formarte, y te servián para razonar y reflexionar sobre el tema, así como también para fomentar una discusión constructiva con los docentes en caso de dudas.


### **Pregunta 1** 

Explica brevemente los detalles relevantes de tu código para entender tu implementación (p.ej., estructura de tu código, funciones, etc.)

*Incluye todas las celdas que consideres oportunas para que sea legible y fácil de seguir.*

<div class="alert alert-block alert-danger">
    <b>NOTA:</b> ¿Cómo me aseguro de que mi implementación es correcta?
    
Es importante que para poder comprobar la correcta implementación de tu solución, tu código tenga en cuenta los siguientes elementos:
-  Mantener en cada iteración la mejor solución encontrada hasta el momento y la iteración en la que se encontró
-  Que puedas imprimir los elementos que se encuentran en la lista tabú
-  Saber en qué iteraciones se llevaron a cabo los reinicios
</div>

<div class="alert alert-block alert-success">
<b>NOTA:</b>  Para verificar tu implementación, debes utilizar el fichero de localizaciones de los 100 concellos gallegos (grafo100cidades.txt). Puedes utilizar como prueba para verificar que la implementaición es correcta inicialmente, el fichero de localizaciones de 8 ciudades gallegas (grafo8cidades.txt). La solución óptima resuelta con una búsqueda informada como A* se situa en torno a los 382km para el problema de prueba con 8 ciudades.
</div>
    
<div class="alert alert-block alert-success">    
<b>AYUDA EXTRA</b>: Si quieres comprobar la implementación de tu práctica puedes probar a utilizar como solución inicial la solución siguiente (pasándosela directamente a la función que implementa el algoritmo de búsqueda tabú):
    
<b> Solución inicial: </b> 
- [77, 9, 43, 73, 2, 53, 12, 83, 92, 33, 50, 63, 54, 59, 64, 74, 55, 14, 35, 5, 58, 87, 37, 7, 69, 79, 89, 21, 23, 80, 20, 56, 75, 68, 27, 95, 78, 25, 88, 51, 47, 91, 49, 60, 13, 36, 70, 42, 11, 22, 40, 72, 28, 97, 19, 71, 29, 90, 85, 76, 16, 24, 81, 84, 34, 8, 31, 38, 67, 45, 44, 32, 96, 10, 61, 94, 17, 18, 93, 30, 52, 66, 99, 26, 46, 39, 15, 86, 41, 4, 62, 1, 48, 82, 57, 98, 3, 65, 6]

Como resultados, deberías obtener los siguientes:
- <b>Mejor solución:</b> [59, 69, 75, 53, 44, 10, 87, 12, 73, 38, 94, 41, 51, 15, 9, 99, 46, 32, 17, 92, 64, 37, 55, 23, 65, 79, 20, 4, 62, 86, 11, 63, 72, 26, 14, 48, 35, 60, 83, 70, 98, 47, 43, 58, 85, 19, 40, 5, 96, 68, 45, 25, 50, 89, 74, 27, 33, 97, 7, 84, 21, 16, 67, 66, 88, 29, 95, 8, 81, 31, 30, 49, 93, 61, 1, 78, 34, 13, 2, 80, 56, 82, 6, 22, 36, 71, 18, 91, 52, 90, 54, 57, 39, 28, 76, 24, 42, 77, 3]
- <b> Coste de la solución: </b> 1663.18
- <b>Iteración en la que se encuentra la mejor solución (empezando a contar en 1):</b> 4633
    
    
    
<b>Prueba adicional con el fichero de grafo50cidadesA.txt</b>:
    
<b> Solución inicial: </b> 
- [25, 6, 43, 23, 13, 47, 4, 11, 46, 41, 15, 38, 21, 26, 14, 42, 49, 32, 33, 3, 12, 20, 8, 48, 39, 28, 37, 45, 36, 2, 17, 9, 31, 29, 7, 24, 1, 5, 18, 35, 44, 22, 16, 30, 34, 10, 40, 19, 27]

Como resultados, deberías obtener los siguientes:
- <b>Mejor solución:</b> [16, 21, 28, 23, 37, 39, 7, 2, 34, 13, 38, 27, 33, 6, 22, 20, 4, 32, 17, 46, 18, 36, 11, 26, 14, 9, 15, 41, 48, 25, 45, 35, 5, 40, 19, 47, 43, 12, 49, 1, 31, 30, 10, 44, 8, 29, 3, 42, 24]
- <b> Coste de la solución: </b> 1032.76
- <b>Iteración en la que se encuentra la mejor solución (empezando a contar en 1):</b> 4280
</div>


## Código

In [17]:
# escribe aqui el codigo de la implementacion basica del algoritmo de busqueda tabu
# se dan pre-definidas las funciones mínimas que debe tener la implementación, dales los parámetros que creas oportunos
# puedes definir funciones adicionales, estos solo son los requisitos mínimos
# recuerda que puedes incluir tantas celdas como quieras

import random
import time

# funcion de generacion de solucion inicial
def genera_solucion_inicial(longitud):
    random.seed(time.time())
    #se obtiene un array con las ciudades intermedias mezcladas
    vector_aleatorio = random.sample(list(range(1, longitud)), longitud-1)
    return vector_aleatorio

def coste_ini(g, solucion):
    suma_coste = 0
    #se van sumando las distancias entre las ciudades de la solucion
    for i in range(0, len(solucion)-1):
        suma_coste += g.distancia(solucion[i], solucion[i+1])
    return suma_coste


# funcion de generacion de vencindario
def genera_vecindario(solucionIni, g, costeIni):
    vecindario={}
    s = solucionIni
    for i in range(len(s)):
        for j in range(i+1, len(s)):
            c = coste_incremental([0]+s+[0], [i+1, j+1], g, costeIni)
            vecindario[c] = [i, j]
    return vecindario

def coste_incremental(solIni, intercambio, g, costeIni):
    difcoste=0
    #restamos los recorridos que rompemos
    difcoste =  - g.distancia(solIni[intercambio[0] - 1], solIni[intercambio[0]])
    difcoste -= g.distancia(solIni[intercambio[1]], solIni[intercambio[1] + 1 ])
    #y sumamos los nuevos
    difcoste += g.distancia(solIni[intercambio[0] - 1], solIni[intercambio[1]])
    difcoste += g.distancia(solIni[intercambio[0]], solIni[intercambio[1] + 1])
    #si las posiciones son seguidas, no hacen falta más operaciones. Si no:
    if (intercambio[0] != (intercambio[1]-1)):
        #restas
        difcoste -= g.distancia(solIni[intercambio[0]], solIni[intercambio[0] + 1])
        difcoste -= g.distancia(solIni[intercambio[1] - 1], solIni[intercambio[1]])
        #sumas
        difcoste += g.distancia(solIni[intercambio[1]], solIni[intercambio[0] + 1])
        difcoste += g.distancia(solIni[intercambio[1] -1], solIni[intercambio[0]])
    return (difcoste + costeIni)

#calcular el coste de todos los posibles vecinos va a ser muy poco eficient
#si sabemos la solucion inicial y su coste
#solamente cambian 4 tramos de la solución (de cambiar 2 posiciones con el elemento de antes y de despues)
#y el resto de distancias siguen iguales
#podríamos restarle al coste inicial, el coste de los segmentos que se van a cambiar
#y sumarle la distancia de los nuevos tramos
#calcular el coste incrementalmente con la solucion inicial, el intercambio que se lleva a cabo, el grafo para poder saber las distancias, y el coste
#el vecindario lo podemos especificar como una lista de intercambios, y el coste de aplicar ese intercambio

def intercambio(s0, indice1, indice2):
    aux = s0[indice1]
    s0[indice1] = s0[indice2]
    s0[indice2] = aux
    return s0


from collections import deque
import numpy as np
import copy

# funcion que implementa el algoritmo búsqueda tabú
def busqueda_tabu(s0, iteraciones=10000, filename='./data/grafo100cidades.txt'):
    maxLT = 100
    INFINITO = 10000000
    
    g = Localizaciones(filename=filename)
    sMejor = []
    sAct = s0
    itActualizacion = 0
    costeMin = coste_ini(g, [0] + sAct + [0])
    fitnessAct = costeMin
    listaTabu = deque()
    iteracion = 0
    reinicios = []
    while (iteracion < iteraciones):
        vecinos = genera_vecindario(sAct, g, fitnessAct)
        fitnessAct = INFINITO #infinito
        #de los vecinos generados, ver que no esté en la lista tabú
        for i in range(len(vecinos.keys())):
            cmin = np.amin(list(vecinos.keys()))
            #si el intercambio con el coste minimo encontrado no está en la lista tabú, nos quedamos con él
            if (vecinos[cmin] not in listaTabu):
                fitnessAct = cmin
                sAct = intercambio(sAct, vecinos[cmin][0], vecinos[cmin][1])
                break; #sale del bucle for
            else: #se borra de vecinos y busca el siguiente mínimo
                del vecinos[cmin]
        if (fitnessAct == INFINITO):
                break; #sale del while pq todos los vecinos generados están en la lista tabú
        #si la solucion obtenida es mejor que la mejor obtenida hasta ahora, se actualiza
        if (fitnessAct < costeMin):
            itActualizacion = iteracion
            sMejor = []
            sMejor = sAct.copy()
            costeMin = fitnessAct
        #REINICIALIZACION
        if (itActualizacion == (iteracion - 1000) or (itActualizacion < (iteracion -1000) and (len(reinicios) > 0) and (reinicios[-1] == (iteracion - 1000)))):
            sAct = sMejor.copy()
            fitnessAct = costeMin
            reinicios.append(iteracion)
            #itActualizacion = iteracion
            #vaciamos la lista tabú
            listaTabu.clear()
        #metemos el nuevo
        listaTabu.append(vecinos[cmin])
        if (len(listaTabu) > maxLT):
            listaTabu.popleft() #quitamos el primer elemento de la lista tabu
        iteracion += 1
    print(reinicios)
    return (sMejor, costeMin, itActualizacion)
#en cada iteracion se genera el mejor vecino de Nrestringido (sin tener en cuenta los moviemientos de la lista tabú)
#diccionario, indexado por el coste para poder sacar de manera eficiente el de menor coste




#comprobacion de funcion coste_incremental
g=Localizaciones(filename='./data/grafo8cidades.txt')
c=coste_incremental([0,1,2,3,4,5,6,7,0], [1,2], g, coste_ini(g, [0,1,2,3,4,5,6,7,0]))
print(c)
print(coste_ini(g, [0,2,1,3,4,5,6,7,0]))
c2 = coste_incremental([0,1,2,3,4,5,6,7,0], [1,4], g, coste_ini(g, [0,1,2,3,4,5,6,7,0]))
print(coste_ini(g, [0,4,2,3,1,5,6,7,0]))
print(c2)


455.35456260967374
455.35456260967374
560.1529486462811
560.1529486462811


In [5]:
tIni = time.time()
sol = busqueda_tabu([25, 6, 43, 23, 13, 47, 4, 11, 46, 41, 15, 38, 21, 26, 14, 42, 49, 32, 33, 3, 12, 20, 8, 48, 39, 28, 37, 45, 36, 2, 17, 9, 31, 29, 7, 24, 1, 5, 18, 35, 44, 22, 16, 30, 34, 10, 40, 19, 27], filename='./data/grafo50cidadesA.txt')
tFin = time.time()
print(sol[1], sol[2]+1)
print('Tiempo ejecución: ', tFin-tIni)

KeyboardInterrupt: 

In [17]:
print(tFin-tIni)

73.0780565738678


In [8]:
#100 CIUDADES
sol100 = busqueda_tabu([77, 9, 43, 73, 2, 53, 12, 83, 92, 33, 50, 63, 54, 59, 64, 74, 55, 14, 35, 5, 58, 87, 37, 7, 69, 79, 89, 21, 23, 80, 20, 56, 75, 68, 27, 95, 78, 25, 88, 51, 47, 91, 49, 60, 13, 36, 70, 42, 11, 22, 40, 72, 28, 97, 19, 71, 29, 90, 85, 76, 16, 24, 81, 84, 34, 8, 31, 38, 67, 45, 44, 32, 96, 10, 61, 94, 17, 18, 93, 30, 52, 66, 99, 26, 46, 39, 15, 86, 41, 4, 62, 1, 48, 82, 57, 98, 3, 65, 6], filename='./data/grafo100cidades.txt')
print(sol100[1])

[1698, 4625, 5632, 6632, 7632, 8632, 9632]
1663.1801510027742


In [11]:
import numpy as np
np.amin([1,2,3,4])

1

###  **Pregunta 2**

La parte experimental de la práctica consiste en realizar **10 ejecuciones diferentes** de la implementación realizada y reportar:
- **Media y desviación** estándar de las soluciones obtenidas.
- El **número de iteración** en el que se obtuvo la mejor solución (por lo que debéis mantener además de la solución óptima hasta el momento $S_{opt}$, la iteración en la que se obtiene). 
- El **tiempo de ejecución** del algoritmo (en las nociones previas se explica cómo hacer esto).

*Incluye todas las celdas que consideres oportunas para que sea legible y fácil de seguir.*


In [3]:
# escribe aquí el código que utilizas para realizar la parte experimental de la práctica
def ejecuciones(numero_ejecuciones=10):
    soluciones = []
    tiempos = []
    iteraciones_sol = []
    for i in range(numero_ejecuciones):
        t_ini = time.time()
        sAux = busqueda_tabu(genera_solucion_inicial(99), iteraciones=5000, filename='./data/grafo100cidades.txt')
        t_final = time.time()
        soluciones.append(sAux[1])
        iteraciones_sol.append(sAux[2])
        tiempos.append(t_final-t_ini)
    return (soluciones, tiempos, iteraciones_sol)

import statistics
def estadisticas(soluciones):
    return (statistics.mean(soluciones), statistics.stdev(soluciones))

In [0]:
experimento = ejecuciones()
stads = estadisticas(experimento[0])

In [32]:
print('Estadísticas de coste: ', stads)
print('Estadísticas de n iteraciones: ', estadisticas(experimento[2]))
print('Tiempo en minutos: ', sum(experimento[1])/60, ' minutos')

Estadísticas de coste:  (1666.2940310421056, 57.80982253171119)
Estadísticas de n iteraciones:  (3815.7, 920.3074425912728)
Tiempo:  29.490019019444784


❗  **NOTA: sé conservador en tu estrategia para verificar tu implementación**, especialmente cuando empleas ficheros de datos grandes como el del problema de las 100 ciudades. Si dejas ejecutando tu algoritmo por un número elevado de iteraciones, puede resultarte útil medir el tiempo que tarda para tomar decisiones sobre donde establecer el límite. 

## P2.2 Implementación de las mejoras (4 pt)

### **Pregunta 3** 

¿Qué intervenciones de mejora te ha llevado a mejores resultados? Prepara un informe en el que explices brevemente las mejoras o intervenciones de mejora realizadas, cómo las has implementado, y diseña un laboratorio para obtener resultados que te permitan explicar porqué las consideras buenas para el problema y soportar tus conclusiones acompañadas de los resultados obtenidos. (MAX. aprox. 1200 palabras)

Aclaraciones: La evaluación de esta parte se llevará a cabo en términos de la completitud y correctitud del laboratorio implementado, así como de la calidad del propio informe, que debe ser conciso y preciso, pudiendo acompañarse de gráficas y tablas que faciliten y fundamenten la explicación e argumentación. Es muy importante explicar de manera clara, precisa y fundamentada. Se valorará más positivamente las intervenciones de mejora que tengan mayor dificultad de implementación que las triviales. Se reservará hasta un punto que se asignará en términos de la calidad de la mejor solución obtenida entre el conjunto de las prácticas entregadas (es por ello que no debes olvidar marcar en tu informe muy claramente cuál ha sido tu mejor solución y con qué configuración/versión).

*Incluye todas las celdas que consideres oportunas para que sea legible y fácil de seguir*

In [0]:
# escribe aqui el codigo de la implementacion optimizada del algoritmo de busqueda tabu
# debe seguir la misma estructuración en funciones que la implementación obligatoria

**Diferentes tipos de generación de la solución inicial**

En la versión incial, la solución inicial la generábamos de manera totalmente aleatoria. La inicialización es un factor importante a la hora de implementar el algoritmo de búsqueda, puesto que puede orientar el recrorrido del algoritmo durante su ejecución. Por eso, en nuestra búsqueda de una mejora, vamos a probar diferentes tipos de generación de la primera solución.

En primer lugar, haremos un árbol de expansión mínimo. Recorriendolo, tendremos la solución inicial a utilizar

In [4]:
#ARBOL EXPANSION MINIMO
import networkx as nx
def generacion_arbol(filename = './data/US120.txt'):
    g = Localizaciones(filename=filename)
    G = nx.Graph()
    #añadimos las ciudades al grafo como nodos
    for i in range(g.nciudades):
        G.add_node(i)
    #añadimos al grafo las distancias entre ciudades (nodos)
    for ciudad1 in range(g.nciudades):
        for ciudad2 in range(ciudad1 + 1, g.nciudades):
            distancia = g.distancia(ciudad1, ciudad2)
            G.add_edge(ciudad1, ciudad2, weight=distancia)
    arbol = nx.minimum_spanning_tree(G)
    sol = list(nx.dfs_preorder_nodes(arbol, source=0))
    return(sol[1:])

In [21]:
tI = time.time()
gen = generacion_arbol()
tF = time.time()
print(gen)
g = Localizaciones(filename='./data/US120.txt')
print('coste: ', coste_ini(g, [0]+gen+[0]))
print(0 in gen) #la solucion no tiene la ciudad inicial 
print('tiempo: ', tF-tI)

[91, 81, 109, 13, 83, 69, 82, 2, 84, 37, 74, 64, 50, 4, 59, 16, 35, 94, 18, 66, 3, 53, 28, 93, 65, 27, 86, 47, 60, 9, 111, 20, 19, 49, 25, 17, 103, 70, 73, 88, 31, 72, 54, 112, 95, 85, 33, 108, 48, 97, 57, 104, 46, 87, 116, 29, 39, 76, 102, 44, 30, 45, 6, 5, 118, 15, 67, 12, 61, 1, 115, 106, 96, 34, 92, 117, 52, 23, 7, 56, 99, 98, 36, 105, 42, 71, 14, 58, 11, 62, 75, 63, 41, 24, 26, 55, 21, 110, 113, 119, 89, 8, 40, 38, 78, 80, 114, 107, 77, 10, 101, 79, 51, 32, 43, 22, 68, 90, 100]
coste:  29275.673071751236
False
tiempo:  0.05643868446350098


Otra manera de generar la solución inicial es la estrategia voraz o greedy. Esata empieza en una ciudad aleatoria y a partir de esta, irá seleccionando la más cercana que todavía no forme parte de la solución.

In [5]:
#INICIALIZACION GREEDY (voraz)
def generacion_greedy(filename='./data/US120.txt'):
    g = Localizaciones(filename=filename)
    long = g.nciudades
    visitadas = [False] * long
    solucion = []
    ciudadAct = 0
    
    for i in range(long-1):
        visitadas[ciudadAct] = True
        solucion.append(ciudadAct)
        
        ciudad_cercana = -1
        distancia_minima = float('inf')

        for ciudad in range(long):
            if not visitadas[ciudad]:
                distancia = g.distancia(ciudadAct, ciudad)
                if distancia < distancia_minima:
                    distancia_minima = distancia
                    ciudad_cercana = ciudad
        
        ciudadAct = ciudad_cercana
    solucion.append(ciudadAct) #la ultima
    
    return solucion[1:]

In [19]:
tI = time.time()
gen = generacion_greedy()
tF = time.time()
print(gen)
g = Localizaciones(filename='./data/US120.txt')
print('coste: ', coste_ini(g, [0]+gen+[0]))
print(0 in gen) #la solucion no tiene la ciudad inicial 
print('tiempo: ', tF-tI)

[91, 81, 109, 119, 89, 8, 40, 38, 105, 36, 42, 4, 59, 16, 35, 94, 18, 66, 3, 53, 50, 64, 74, 37, 84, 2, 69, 83, 82, 13, 78, 80, 114, 107, 77, 10, 101, 79, 51, 22, 68, 90, 100, 43, 32, 71, 110, 113, 14, 58, 11, 62, 75, 63, 41, 24, 21, 55, 28, 93, 65, 27, 86, 47, 60, 9, 111, 20, 49, 25, 17, 92, 34, 117, 52, 23, 103, 96, 70, 73, 88, 31, 72, 54, 112, 95, 85, 33, 108, 48, 97, 57, 6, 45, 30, 102, 44, 76, 39, 29, 116, 87, 5, 118, 15, 67, 12, 61, 115, 106, 1, 98, 99, 56, 7, 19, 104, 46, 26]
coste:  27033.84988200304
False
tiempo:  0.0178682804107666


Otra manera de generar la solucion inicial es generar varias aleatorias de la misma manera que en la implementación inicial y quedarnos con la mejor de estas

In [6]:
def generacion_mejor_aleatorias(filename='./data/US120.txt'):
    g = Localizaciones(filename=filename)
    coste = float('inf')
    sol = []
    for i in range(15):
        aux = genera_solucion_inicial(g.nciudades)
        cAux = coste_ini(g, aux)
        if (cAux < coste):
            coste = cAux
            sol = aux
    return sol

In [22]:
tI = time.time()
gen = generacion_mejor_aleatorias()
tF = time.time()
print(gen)
g = Localizaciones(filename='./data/US120.txt')
print('coste: ', coste_ini(g, [0]+gen+[0]))
print('tiempo: ', tF-tI)

[65, 103, 25, 116, 1, 115, 95, 38, 18, 16, 59, 6, 108, 46, 9, 73, 72, 39, 3, 91, 30, 5, 27, 89, 7, 69, 113, 13, 8, 85, 44, 96, 14, 82, 10, 15, 63, 47, 53, 77, 81, 104, 40, 48, 68, 41, 64, 35, 4, 20, 100, 36, 75, 37, 19, 102, 71, 31, 58, 107, 26, 34, 21, 118, 76, 60, 70, 88, 62, 111, 99, 87, 105, 114, 109, 94, 12, 54, 80, 2, 97, 78, 29, 61, 79, 28, 32, 42, 83, 66, 55, 52, 67, 86, 33, 23, 106, 74, 49, 93, 112, 45, 98, 24, 17, 117, 43, 11, 101, 22, 119, 90, 57, 110, 84, 50, 51, 92, 56]
coste:  171503.71065139954
tiempo:  0.020794391632080078


In [12]:
ti_arbol = time.time()
sol_arbol = busqueda_tabu(generacion_arbol(), iteraciones=5000, filename='./data/US120.txt')
tf_arbol = time.time()

ti_greedy = time.time()
sol_greedy = busqueda_tabu(generacion_greedy(), iteraciones=5000, filename='./data/US120.txt')
tf_greedy = time.time()

ti_aleats = time.time()
sol_aleats = busqueda_tabu(generacion_mejor_aleatorias(),iteraciones=5000, filename='./data/US120.txt')
tf_aleats = time.time()

print('Tipo --- Tiempos --- Costes --- itearcion')
print('Arbol: ', (tf_arbol-ti_arbol)/60, sol_arbol[1], sol_arbol[2])
print('Greedy: ', (tf_greedy-ti_greedy)/60, sol_greedy[1], sol_greedy[2])
print('Aleats: ', (tf_aleats-ti_aleats)/60, sol_aleats[1], sol_aleats[2])
print('\n')

[1979, 2997, 4002]


[1361, 2383, 3387, 4387]


[]
Tipo --- Tiempos --- Costes --- itearcion
Arbol:  5.652783421675364 26201.040813337862 4992
Greedy:  5.636741471290589 25619.983016850812 2387
Aleats:  5.3550938685735066 34811.78294222459 4142




Tras observar estos resultados podemos ver que la solución greedy es la que obtiene los mejores resultados en un menor número de iteraciones, por lo que se podría reducir el número de estas sin tener que prescindir de un resultado mucho mejor.  A partir de aquí, utilizaremos el algoritmo greedy como generador de la solución inicial de las mejoras implementadas

**Gestión de la lista tabú**

En cuanto a la gestión de la lista tabú, en la primera implementación excluíamos todas las soluciones generadas que estuviesen en la lista tabú. Otra opción es permitir excepciones, para hacerla más flexible. De esta manera ,podríamos aceptar determinadas soluciones si cumplen unas determinadas condiciones, los llamados criterios de aspiración. Esto podría ser por ejemplo, aceptar una solución si su coste es menor que el de la mejor solución encontrada hasta ese momento.

In [7]:
# acepta soluciones de la lista tabú mejores que la actual
def busqueda_tabu_m1(s0, iteraciones=10000, filename='./data/US120.txt'):
    maxLT = 100
    INFINITO = 100000000
    
    g = Localizaciones(filename=filename)
    sMejor = []
    sAct = s0
    itActualizacion = 0
    costeMin = coste_ini(g, [0] + sAct + [0])
    fitnessAct = costeMin
    listaTabu = deque()
    iteracion = 0
    reinicios = []
    while (iteracion < iteraciones):
        vecinos = genera_vecindario(sAct, g, fitnessAct)
        fitnessAct = INFINITO #infinito
        #de los vecinos generados, ver que no esté en la lista tabú
        for i in range(len(vecinos.keys())):
            cmin = np.amin(list(vecinos.keys()))
            #si el intercambio con el coste minimo encontrado no está en la lista tabú, nos quedamos con él
            if (vecinos[cmin] not in listaTabu or (cmin < costeMin)):
                fitnessAct = cmin
                sAct = intercambio(sAct, vecinos[cmin][0], vecinos[cmin][1])
                break; #sale del bucle for
            else: #se borra de vecinos y busca el siguiente mínimo
                del vecinos[cmin]
        if (fitnessAct == INFINITO):
                break; #sale del while pq todos los vecinos generados están en la lista tabú
        #si la solucion obtenida es mejor que la mejor obtenida hasta ahora, se actualiza
        if (fitnessAct < costeMin):
            itActualizacion = iteracion
            sMejor = []
            sMejor = sAct.copy()
            costeMin = fitnessAct
        #REINICIALIZACION
        if (itActualizacion == (iteracion - 1000) or (itActualizacion < (iteracion -1000) and (len(reinicios) > 0) and (reinicios[-1] == (iteracion - 1000)))):
            sAct = sMejor.copy()
            fitnessAct = costeMin
            reinicios.append(iteracion)
            #itActualizacion = iteracion
            #vaciamos la lista tabú
            listaTabu.clear()
        #metemos el nuevo
        listaTabu.append(vecinos[cmin])
        if (len(listaTabu) > maxLT):
            listaTabu.popleft() #quitamos el primer elemento de la lista tabu
        iteracion += 1
    print(reinicios)
    return (sMejor, costeMin, itActualizacion)

In [14]:
tim1 = time.time()
sm1 = busqueda_tabu_m1(generacion_greedy(), iteraciones=5000)
tfm1 = time.time()
print('m1: ', (tfm1-tim1)/60, sm1[1], sm1[2])


[1703, 2703, 3703, 4703]
m1:  5.536862647533416 25926.32317122482 703


Aceptando soluciones mejores podemos ver que encuentra la mejor solución en muy pocas iteraciones. Además, la solución es peor. Esto nos indica que el algoritmo cae en un mínimo local, por lo que no vamos a contemplar esta modificación

**Generación de vecinos**

Hay muchas maneras diferentes de generar el vecindario a partir de la solución actual. En la primera implementación se utilizaba el operador de intercambio para generar todos los posibles vecinos. 

A continuación probaremos con el mismo operador pero generando la mitad de los vecinos, y con el operador de inserción. Para este último tendremos que cambiar también la manera de insertar los movimientos en la tabla tabú. La generación de vecinos nos devolverá un vecindario en forma de diccionario donde las claves serán los costes, y los valores una tupla [i, j] donde i será el índice actual de la ciudad a mover, y j será la nueva posición en la que se insertará. Por esta razón, incluiremos en la lista tabú la tupla al revés, [j, i], de manera que el movimiento que se prohibirá es que la ciudad de la posición j (la que se acaba de insertar) vuelva a la posición i.

In [8]:
def generacion_mitad_vecinos(solucionIni, g, costeIni):
    vecindario={}
    s = solucionIni
    for i in range(len(s)):
        for j in range(i+1,len(s), 2):
            c = coste_incremental([0]+s+[0], [i+1, j+1], g, costeIni)
            vecindario[c] = [i, j]
    return vecindario

g = Localizaciones(filename='./data/US120.txt')
s = generacion_greedy()
print(len(generacion_mitad_vecinos(s, g, coste_ini(g, s))))


NameError: name 'Localizaciones' is not defined

In [9]:
#generando la mitad de los vecinos
def busqueda_tabu_m2(s0, iteraciones=10000, filename='./data/US120.txt'):
    maxLT = 100
    INFINITO = 100000000
    
    g = Localizaciones(filename=filename)
    sMejor = []
    sAct = s0
    itActualizacion = 0
    costeMin = coste_ini(g, [0] + sAct + [0])
    fitnessAct = costeMin
    listaTabu = deque()
    iteracion = 0
    reinicios = []
    while (iteracion < iteraciones):
        vecinos = generacion_mitad_vecinos(sAct, g, fitnessAct)
        fitnessAct = INFINITO #infinito
        #de los vecinos generados, ver que no esté en la lista tabú
        for i in range(len(vecinos.keys())):
            cmin = np.amin(list(vecinos.keys()))
            #si el intercambio con el coste minimo encontrado no está en la lista tabú, nos quedamos con él
            if (vecinos[cmin] not in listaTabu):
                fitnessAct = cmin
                sAct = intercambio(sAct, vecinos[cmin][0], vecinos[cmin][1])
                break; #sale del bucle for
            else: #se borra de vecinos y busca el siguiente mínimo
                del vecinos[cmin]
        if (fitnessAct == INFINITO):
                break; #sale del while pq todos los vecinos generados están en la lista tabú
        #si la solucion obtenida es mejor que la mejor obtenida hasta ahora, se actualiza
        if (fitnessAct < costeMin):
            itActualizacion = iteracion
            sMejor = []
            sMejor = sAct.copy()
            costeMin = fitnessAct
        #REINICIALIZACION
        if (itActualizacion == (iteracion - 1000) or (itActualizacion < (iteracion -1000) and (len(reinicios) > 0) and (reinicios[-1] == (iteracion - 1000)))):
            sAct = sMejor.copy()
            fitnessAct = costeMin
            reinicios.append(iteracion)
            #itActualizacion = iteracion
            #vaciamos la lista tabú
            listaTabu.clear()
        #metemos el nuevo
        listaTabu.append(vecinos[cmin])
        if (len(listaTabu) > maxLT):
            listaTabu.popleft() #quitamos el primer elemento de la lista tabu
        iteracion += 1
    print(reinicios)
    return (sMejor, costeMin, itActualizacion)

In [84]:
ti_m2 = time.time()
sol_m2 = busqueda_tabu_m2(generacion_greedy(), iteraciones=5000)
tf_m2 = time.time()


print('Tipo --- Tiempos --- Costes --- itearcion')
print('generar mitad vecinos: ', (tf_m2-ti_m2)/60, sol_m2[1], sol_m2[2])
print('\n')

[1011, 2013, 3013, 4013]
Tipo --- Tiempos --- Costes --- itearcion
generar mitad vecinos:  2.5667369246482847 26535.1370152978 1013




Aquí podemos ver que generando la mitad de los vecinos el tiempo de computación es indudablemente más bajo que en la original, puesto que se exploran muchas menos opciones. Además, encuentra la mejor solución muy pronto y el coste se aleja bastante de la mejor solución encontrada hasta el momento,  lo que nos indica que cayó en un mínimo local, por lo que no seguiremos explorando esta opción de generación de vecinos.

In [10]:
def coste_insercion(solIni, insercion, g, costeIni):
    difcoste = 0
    #restamos los recorridos que rompemos
    difcoste -= g.distancia(solIni[insercion[0]-1], solIni[insercion[0]])
    difcoste -= g.distancia(solIni[insercion[0]+1], solIni[insercion[0]])
    #sumamos los nuevos
    difcoste += g.distancia(solIni[insercion[0]], solIni[insercion[1]])
    difcoste += g.distancia(solIni[insercion[0]-1], solIni[insercion[0]+1])
    if (insercion[0] < insercion[1]):
        #restamos el punto donde se va a insertar la ciudad
        difcoste -= g.distancia(solIni[insercion[1]], solIni[insercion[1]+1])
        difcoste += g.distancia(solIni[insercion[1]+ 1], solIni[insercion[0]])
    else:
        #restamos el punto donde se va a insertar la ciudad
        difcoste -= g.distancia(solIni[insercion[1]-1], solIni[insercion[1]])
        difcoste += g.distancia(solIni[insercion[1]-1], solIni[insercion[0]])
    return (difcoste + costeIni)

def generacion_insercion(sol, g, costeIni):
    vecindario = {}
    s = sol
    for i in range(len(s)):
        for j in range(len(s)):
            if (i != j) and (i != j+1):
                # Inserta la ciudad i en la posición j
                c = coste_insercion([0]+s+[0], [i+1, j+1], g, costeIni)
                #i es la posicion inicial de la ciudad cambiada y j es la posicion de inserción
                vecindario[c] = [i, j]
    return vecindario


def insercion(sAct, posIni, posInsercion):
    sol = sAct
    aux = sol[posIni]
    sol.pop(posIni)
    sol.insert(posInsercion, aux)
    return sol

def busqueda_tabu_insercion(s0, iteraciones=10000, filename='./data/US120.txt'):
    maxLT = 100
    INFINITO = 10000000
    
    g = Localizaciones(filename=filename)
    sMejor = s0
    sAct = s0
    itActualizacion = 0
    costeMin = coste_ini(g, [0] + s0 + [0])
    fitnessAct = costeMin
    listaTabu = deque()
    iteracion = 0
    reinicios = []
    while (iteracion < iteraciones):
        vecinos = generacion_insercion(sAct, g, fitnessAct)
        fitnessAct = INFINITO #infinito
        #de los vecinos generados, ver que no esté en la lista tabú
        for i in range(len(vecinos.keys())):
            cmin = np.amin(list(vecinos.keys()))
            #si el intercambio con el coste minimo encontrado no está en la lista tabú, nos quedamos con él
            if (vecinos[cmin] not in listaTabu):
                fitnessAct = cmin
                sAct = insercion(sAct, vecinos[cmin][0], vecinos[cmin][1])
                break; #sale del bucle for
            else: #se borra de vecinos y busca el siguiente mínimo
                del vecinos[cmin]
        if (fitnessAct == INFINITO):
                break; #sale del while pq todos los vecinos generados están en la lista tabú
        #si la solucion obtenida es mejor que la mejor obtenida hasta ahora, se actualiza
        if (fitnessAct < costeMin):
            itActualizacion = iteracion
            sMejor = []
            sMejor = sAct.copy()
            costeMin = fitnessAct
        #REINICIALIZACION
        if (itActualizacion == (iteracion - 1000) or (itActualizacion < (iteracion -1000) and (len(reinicios) > 0) and (reinicios[-1] == (iteracion - 1000)))):
            sAct = sMejor.copy()
            fitnessAct = costeMin
            reinicios.append(iteracion)
            #itActualizacion = iteracion
            #vaciamos la lista tabú
            listaTabu.clear()
        #metemos el nuevo
        listaTabu.append([vecinos[cmin][1], vecinos[cmin][0]]) #cambiamos la manera de meterlo en la lista tabú 
        if (len(listaTabu) > maxLT):
            listaTabu.popleft() #quitamos el primer elemento de la lista tabu
        iteracion += 1
    print(reinicios)
    return (sMejor, costeMin, itActualizacion)
#en cada iteracion se genera el mejor vecino de Nrestringido (sin tener en cuenta los moviemientos de la lista tabú)
#diccionario, indexado por el coste para poder sacar de manera eficiente el de menor coste


In [87]:
ti_insercion = time.time()
sol_insercion = busqueda_tabu_insercion(generacion_greedy(), iteraciones = 5000)
tf_insercion = time.time()


print('Tipo --- Tiempos --- Costes --- itearcion')
print('inserc ', (tf_insercion-ti_insercion)/60, sol_insercion[1], sol_insercion[2])
print(sol_insercion[0])

[1027, 2027, 3027, 4027]
Tipo --- Tiempos --- Costes --- itearcion
inserc  6.910066866874695 24673.721028278356 27
[91, 81, 109, 119, 89, 40, 38, 105, 36, 4, 59, 16, 35, 94, 18, 66, 3, 53, 50, 74, 37, 84, 82, 69, 83, 78, 80, 114, 107, 77, 51, 79, 101, 10, 22, 68, 100, 90, 43, 32, 8, 71, 42, 64, 58, 62, 75, 21, 55, 28, 93, 65, 27, 86, 47, 60, 9, 20, 111, 25, 17, 92, 34, 49, 70, 73, 88, 31, 72, 54, 112, 95, 85, 33, 108, 48, 97, 57, 6, 45, 30, 44, 102, 76, 39, 29, 116, 87, 5, 118, 15, 67, 12, 115, 106, 1, 61, 98, 99, 56, 7, 19, 104, 46, 96, 103, 117, 52, 23, 41, 63, 24, 26, 11, 14, 113, 110, 2, 13]


**Estrategias de reinicialización**

**INTENSIFICACIÓN**

Como estrategia de reinicialización inicialmente tilizamos una estrategia por intensificación en la que volvíamos a la mejor solución encontrada hasta el momento y seguíamos explorando desde esa con la lista tabú vacía. A parte de esta, hay otras estrategias d reinicialización por intensificación como en lugar de escoger la mejor solución encontrada, almacenar las x mejores, en este caso contemplaremos 10, y de estas, elegir una aleatoriamente para reiniciar el algoritmo y la quitaremos de esta lista. Con respecto a la memoria de corto plazo, vaciaremos la lista tabú a 0 para que se puedan explorar nuevas soluciones.

In [65]:
g = Localizaciones(filename='./data/grafo8cidades.txt')
print(coste_ini(g, [0]+[7, 6, 2,4,  1, 3, 5]+[0]))

664.9266099934867


In [11]:
#con reinicialización por intensificación. Aleatoria de las 10 mejores
def busqueda_tabu_m3(s0, tamMejores=10, iteraciones=10000, filename='./data/US120.txt'):
    maxLT = 100
    INFINITO = 10000000
    
    g = Localizaciones(filename=filename)
    sMejor = []
    sMejores = {}
    sAct = s0
    itActualizacion = 0
    costeMin = coste_ini(g, [0] + sAct + [0])
    fitnessAct = costeMin
    sMejores[costeMin] = s0
    listaTabu = deque()
    iteracion = 0
    reinicios = []
    while (iteracion < iteraciones):
        vecinos = genera_vecindario(sAct, g, fitnessAct)
        fitnessAct = INFINITO #infinito
        #de los vecinos generados, ver que no esté en la lista tabú
        for i in range(len(vecinos.keys())):
            cmin = np.amin(list(vecinos.keys()))
            #si el intercambio con el coste minimo encontrado no está en la lista tabú, nos quedamos con él
            if (vecinos[cmin] not in listaTabu):
                fitnessAct = cmin
                sAct = intercambio(sAct, vecinos[cmin][0], vecinos[cmin][1])
                if (len(sMejores) < 10):
                    sMejores[fitnessAct] = sAct
                else:
                    maxm = np.amax(list(sMejores.keys()))
                    if (fitnessAct < maxm):
                        del sMejores[maxm]
                        sMejores[fitnessAct] = sAct
                break; #sale del bucle for
            else: #se borra de vecinos y busca el siguiente mínimo
                del vecinos[cmin]
        if (fitnessAct == INFINITO):
                break; #sale del while pq todos los vecinos generados están en la lista tabú
        #si la solucion obtenida es mejor que la mejor obtenida hasta ahora, se actualiza
        if (fitnessAct < costeMin):
            itActualizacion = iteracion
            sMejor = []
            sMejor = sAct.copy()
            costeMin = fitnessAct
        #REINICIALIZACION
        if (itActualizacion == (iteracion - 1000) or (itActualizacion < (iteracion -1000) and (len(reinicios) > 0) and (reinicios[-1] == (iteracion - 1000)))):
            #elegimos una solucion aleatoria de las 10 mejores
            key = random.choice(list(sMejores.keys()))
            sAct = sMejores.pop(key)
            fitnessAct = costeMin
            reinicios.append(iteracion)
            #actualizamos la lista tabu
            listaTabu.clear()
        #metemos el nuevo
        listaTabu.append(vecinos[cmin])
        if (len(listaTabu) > maxLT):
            listaTabu.popleft() #quitamos el primer elemento de la lista tabu
        iteracion += 1
    print(reinicios)
    return (sMejor, costeMin, itActualizacion)

In [80]:
ti_intensf = time.time()
sol_intensf = busqueda_tabu_m3(generacion_greedy(), iteraciones = 5000)
tf_intensf = time.time()


print('Tipo --- Tiempos --- Costes --- itearcion')
print('inserc ', (tf_intensf-ti_intensf)/60, sol_intensf[1], sol_intensf[2])
print(sol_intensf[0])

[1361, 3007, 4030]
Tipo --- Tiempos --- Costes --- itearcion
inserc  4.9284229675928755 23806.746757341924 4067
[81, 91, 109, 13, 119, 89, 8, 38, 40, 71, 42, 105, 36, 4, 35, 59, 16, 18, 66, 3, 53, 50, 64, 74, 37, 84, 82, 83, 78, 77, 101, 10, 79, 51, 32, 43, 90, 100, 68, 22, 114, 107, 80, 69, 2, 110, 113, 14, 58, 11, 26, 24, 41, 63, 75, 62, 55, 21, 28, 93, 65, 27, 86, 47, 60, 9, 20, 111, 49, 25, 17, 34, 92, 23, 52, 117, 103, 96, 70, 73, 88, 31, 72, 54, 112, 95, 85, 33, 108, 48, 97, 57, 6, 45, 30, 44, 102, 76, 39, 29, 116, 87, 5, 118, 15, 67, 12, 61, 115, 106, 1, 99, 98, 19, 46, 104, 7, 56, 94]


**DIVERSIFICACIÓN**

La estrategia de reinicialización busca explorar más el espacio de soluciones. Para hacer esto, a la hora de reinicializar, en lugar de partir de la mejor solución encontrada hasta el momento, partiremos de una solución generada aleatoriamente, en concreto con la función anterior con la que seleccionamos la solución aleatoria con menor coste de 5 generadas.

In [13]:
#con reinicialización por diversificacion. Mejor de 5 aleatorias
def busqueda_tabu_m4(s0, tamMejores=10, iteraciones=10000, filename='./data/US120.txt'):
    maxLT = 100
    INFINITO = 10000000
    
    g = Localizaciones(filename=filename)
    sMejor = []
    sMejores = {}
    sAct = s0
    itActualizacion = 0
    costeMin = coste_ini(g, [0] + sAct + [0])
    fitnessAct = costeMin
    sMejores[costeMin] = s0
    listaTabu = deque()
    iteracion = 0
    reinicios = []
    while (iteracion < iteraciones):
        vecinos = genera_vecindario(sAct, g, fitnessAct)
        fitnessAct = INFINITO #infinito
        #de los vecinos generados, ver que no esté en la lista tabú
        for i in range(len(vecinos.keys())):
            cmin = np.amin(list(vecinos.keys()))
            #si el intercambio con el coste minimo encontrado no está en la lista tabú, nos quedamos con él
            if (vecinos[cmin] not in listaTabu):
                fitnessAct = cmin
                sAct = intercambio(sAct, vecinos[cmin][0], vecinos[cmin][1])
                if (len(sMejores) < 10):
                    sMejores[fitnessAct] = sAct
                else:
                    maxm = np.amax(list(sMejores.keys()))
                    if (fitnessAct < maxm):
                        del sMejores[maxm]
                        sMejores[fitnessAct] = sAct
                break; #sale del bucle for
            else: #se borra de vecinos y busca el siguiente mínimo
                del vecinos[cmin]
        if (fitnessAct == INFINITO):
                break; #sale del while pq todos los vecinos generados están en la lista tabú
        #si la solucion obtenida es mejor que la mejor obtenida hasta ahora, se actualiza
        if (fitnessAct < costeMin):
            itActualizacion = iteracion
            sMejor = []
            sMejor = sAct.copy()
            costeMin = fitnessAct
        #REINICIALIZACION
        if (itActualizacion == (iteracion - 1000) or (itActualizacion < (iteracion -1000) and (len(reinicios) > 0) and (reinicios[-1] == (iteracion - 1000)))):
            #elegimos una solucion aleatoria de las 10 mejores
            sAct = generacion_mejor_aleatorias()
            fitnessAct = coste_ini(g, [0] + sAct + [0])
            reinicios.append(iteracion)
            #actualizamos la lista tabu
            listaTabu.clear()
        #metemos el nuevo
        listaTabu.append(vecinos[cmin])
        if (len(listaTabu) > maxLT):
            listaTabu.popleft() #quitamos el primer elemento de la lista tabu
        iteracion += 1
    print(reinicios)
    return (sMejor, costeMin, itActualizacion)

In [18]:
import time
ti_diversif= time.time()
sol_diversif = busqueda_tabu_m4(generacion_greedy(), iteraciones = 5000)
tf_diversif = time.time()


print('Tipo --- Tiempos --- Costes --- itearcion')
print('diversificacion ', (tf_diversif-ti_diversif)/60, sol_diversif[1], sol_diversif[2])
print(sol_diversif[0])

[1361, 2361, 3361, 4361]
Tipo --- Tiempos --- Costes --- itearcion
diversificacion  4.6967650334040325 26302.462686658382 361
[81, 91, 109, 119, 89, 8, 40, 38, 71, 42, 105, 36, 4, 35, 18, 59, 16, 66, 3, 53, 50, 64, 74, 37, 84, 69, 82, 83, 13, 78, 80, 107, 114, 90, 100, 68, 22, 32, 43, 79, 51, 10, 101, 77, 2, 110, 113, 14, 58, 11, 26, 24, 41, 63, 75, 62, 55, 21, 28, 93, 65, 27, 86, 47, 60, 20, 111, 9, 49, 25, 17, 34, 92, 23, 52, 117, 103, 96, 70, 73, 88, 31, 72, 54, 112, 95, 85, 33, 108, 48, 97, 57, 6, 45, 30, 102, 44, 76, 39, 29, 116, 87, 5, 118, 15, 67, 12, 115, 106, 1, 61, 98, 99, 56, 7, 19, 46, 104, 94]


### Pregunta 3

En esta pregunta llevaremos a cabo la implementación de diferentes variaciones del algoritmo de búsqueda tabú con la finalidad de encontrar algunas con las que obtengamos alguna mejora, ya sea en términos de tiempo como de los resultados obtenidos. Para esto modificaremos diferentes parámetros y operaciones como el modo de generación de la solución inicial, el modo de gestió de la lista tabú,  o el operador de generación de vecinos, entre otros. Estas mejoras las probaremos con el archivo de 120 ciudades.

**Generación de la solución inicial**

En lo tocante a los diferentes tipos de generación de la solución inicial, en la primera versión implementada, esta la generábamos de manera totalmente aleatoria. La inicialización es un factor importante a la hora de implementar un algoritmo de búsqueda, puesto que puede orientar el recrorrido del algoritmo durante su ejecución. Por eso, en nuestra búsqueda de una mejora, vamos a probar diferentes tipos de generación de la primera solución.

En primer lugar, haremos un árbol de expansión mínimo. Recorriendolo en profundidad tendremos la solución inicial a utilizar. También probaremos con la estrategia voraz o greedy. Esta empieza en una ciudad aleatoria y a partir de esta, irá seleccionando la más cercana que todavía no forme parte de la solución. Otra manera puede ser también la generación de un número controlado de posibles soluciones aleatorias y elegir la de menor coste de estas.

Tras implementar y probar estas tres opciones podemos ver que el árbol de expansión mínimo es el que más tarda en ejecutarse por si solo. Las otras dos necesitan de poco tiempo hasta dar con la solución qeue utilizaremos, sin embargo, la aleatoria puede dar resultados de un coste muy elevado.

In [24]:
from tabulate import tabulate

encabezados=['Tipo', 'Coste', 'Tiempo (seg)']
datos=[['Árbol expasión', 29275.673071751236, 0.05643868446350098],
      ['Greedy', 27033.84988200304, 0.0178682804107666],
      ['Aleatorias', 171503.71065139954, 0.020794391632080078]]
tabla = tabulate(datos, headers=encabezados, tablefmt="pretty")
print(tabla)

+----------------+--------------------+----------------------+
|      Tipo      |       Coste        |     Tiempo (seg)     |
+----------------+--------------------+----------------------+
| Árbol expasión | 29275.673071751236 | 0.05643868446350098  |
|     Greedy     | 27033.84988200304  |  0.0178682804107666  |
|   Aleatorias   | 171503.71065139954 | 0.020794391632080078 |
+----------------+--------------------+----------------------+


Además, en cuanto al comportamiento del algoritmo con la solución inicial generada de estas maneras, obtenemos que mediante el algoritmo voraz obtenemos el resultado final con un coste más reducido, además de que necesita menos iteraciones que para los otros casos, por lo que se podría reducir el número de estas sin tener que prescindir de un buen resultado. 

<img src=".Practica2_BusquedaTabu.ipynb.upload/paste-0.4311559410605462" style="max-width:100%" />

A partir de aquí, utilizaremos el algoritmo greedy como generador de la solución inicial de las mejoras implementadas.

**Gestión de la lista tabú**

En cuanto a la gestión de la lista tabú, en la primera implementación excluíamos todas las soluciones generadas que estuviesen en la lista tabú. Otra opción es permitir excepciones, para hacerla más flexible. De esta manera, podríamos aceptar determinadas soluciones si cumplen unas ciertas condiciones, llamadas criterios de aspiración. Esto podría ser por ejemplo, aceptar una solución si su coste es menor que el de la mejor solución encontrada hasta ese momento.

Aceptando soluciones mejores podemos ver que encuentra la mejor solución en muy pocas iteraciones. Además, la solución es peor. Esto nos indica que el algoritmo cae en un mínimo local, por lo que no vamos a contemplar esta modificación

In [25]:
from tabulate import tabulate

encabezados=['Tipo', 'Coste', 'Tiempo (min)', 'Iteraciones']
datos=[['Aceptación tabús mejores', 25926.32317122482, 5.536862647533416, 703],
      ['Generación menos vecinos', 26535.1370152978, 2.5667369246482847, 1013],
      ['Inserción', 24673.721028278356, 6.910066866874695, 27]]
tabla = tabulate(datos, headers=encabezados, tablefmt="pretty")
print(tabla)

+--------------------------+--------------------+--------------------+-------------+
|           Tipo           |       Coste        |    Tiempo (min)    | Iteraciones |
+--------------------------+--------------------+--------------------+-------------+
| Aceptación tabús mejores | 25926.32317122482  | 5.536862647533416  |     703     |
| Generación menos vecinos |  26535.1370152978  | 2.5667369246482847 |    1013     |
|        Inserción         | 24673.721028278356 | 6.910066866874695  |     27      |
+--------------------------+--------------------+--------------------+-------------+


**Generación de vecinos**

Hay muchas maneras diferentes de generar el vecindario a partir de la solución actual. En la primera implementación se utilizaba el operador de intercambio para generar todos los posibles vecinos. 

A continuación probaremos con el mismo operador pero generando la mitad de los vecinos, y con el operador de inserción. Para este último tendremos que cambiar también la manera de insertar los movimientos en la tabla tabú. La generación de vecinos nos devolverá un vecindario en forma de diccionario donde las claves serán los costes, y los valores una tupla [i, j] donde i será el índice actual de la ciudad a mover, y j será la nueva posición en la que se insertará. Por esta razón, incluiremos en la lista tabú la tupla al revés, [j, i], de manera que el movimiento que se prohibirá es que la ciudad de la posición j (la que se acaba de insertar) vuelva a la posición i.

En la tabla superior podemos ver que generando la mitad de los vecinos el tiempo de computación es indudablemente más bajo que en la original, puesto que se exploran muchas menos opciones. Además, encuentra la mejor solución muy pronto y el coste se aleja bastante de la mejor solución encontrada hasta el momento,  lo que nos indica que cayó en un mínimo local, por lo que no seguiremos explorando esta opción de generación de vecinos.

Por otra parte, en cuanto a la inserción como operador , consigue una solución de coste bajo, pero tarda bastante más tiempo que la versión original. Además, la solución devuelta se encuentra en las primeras iteraciones, lo que nos puede hacer entender que se queda estancado en un mínimo local.

**Estrategias de reinicialización**

**INTENSIFICACIÓN**

Como estrategia de reinicialización inicialmente empleamos una estrategia por intensificación en la que volvíamos a la mejor solución encontrada hasta el momento y seguíamos explorando desde esa con la lista tabú vacía. A parte de esta, hay otras estrategias d reinicialización por intensificación como en lugar de escoger la mejor solución encontrada, almacenar las x mejores, en este caso contemplaremos 10, y de estas, elegir una aleatoriamente para reiniciar el algoritmo y la quitaremos de esta lista. Con respecto a la memoria de corto plazo, vaciaremos la lista tabú a 0 para que se puedan explorar nuevas soluciones.

**DIVERSIFICACIÓN**

La estrategia de reinicialización busca explorar más el espacio de soluciones. Para hacer esto, a la hora de reinicializar, en lugar de partir de la mejor solución encontrada hasta el momento, partiremos de una solución generada aleatoriamente, en concreto con la función anterior con la que seleccionamos la solución aleatoria con menor coste de 5 generadas.

Tras probar con estas dos variaciones podemos ver que con la estrategia de intensificación obtenemos el mejor resultado obtenido hasta el momento. Además, ninguna de estas dos variaciones aumenta excesivamente el tiempo de computacionalmente.

In [26]:
from tabulate import tabulate

encabezados=['Tipo', 'Coste', 'Tiempo (min)', 'Iteraciones']
datos=[['Intensificación', 23806.746757341924, 4.9284229675928755, 4067],
      ['Diversificación', 26302.462686658382, 4.6967650334040325, 361]]
tabla = tabulate(datos, headers=encabezados, tablefmt="pretty")
print(tabla)

+-----------------+--------------------+--------------------+-------------+
|      Tipo       |       Coste        |    Tiempo (min)    | Iteraciones |
+-----------------+--------------------+--------------------+-------------+
| Intensificación | 23806.746757341924 | 4.9284229675928755 |    4067     |
| Diversificación | 26302.462686658382 | 4.6967650334040325 |     361     |
+-----------------+--------------------+--------------------+-------------+


Tras la realización de estos experimentos, de las variaciones implementadas, la combinación que mejores resultados nos brinda es:
La generación de la solución inicial mediante un algoritmo voraz o greedy.La gestión de la lista tabú de manera normal, puesto que al aceptar soluciones tabú mejores caía fácilmente en mínimos locales.
Para la generación de vecinos, en caso de querer disminuir el tiempo de computación podemos optar por la generación de la mitad de los vecinos, o bien, de preferir una solución más óptima, la generación medianete el operador de intercambio de dos ciudados da buenos resultados en un tiempo asumible.
Por último, para las estrategias de reinicialización, la mejor sería la de intensificación, puesto que con ella, tras un período de exploración (hasta que no es necesaria una reinicialización), aporta explotación de las mejores soluciones encontradas, de manera que podemos encontrar algunas aun mejores a partir de estas.